# First Step
This first notebook takes the sections already parsed into .mmd files by the nougat library to generate summarized sections.

In [ ]:
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer
from peft import PeftModel
import torch

def read_mmd_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    return lines

def split_mmd_into_sections(lines):
    sections = {}
    current_section = ''
    current_text = ''

    for line in lines:
        if line.startswith('#'):
            if current_section:
                sections[current_section] = current_text.strip()
                current_text = ''
            current_section = line.strip()
        elif line.startswith('##'):
            if current_text:
                current_text += '\n'
            current_text += line.strip()
        else:
            current_text += '\n' + line.strip()

    if current_section:
        sections[current_section] = current_text.strip()

    return sections

In [ ]:
base_model= "meta-llama/Meta-Llama-3-8B"
use_perf = False
paper_path = './3/3.mmd'

tokenizer = LlamaTokenizer.from_pretrained(base_model)
model = LlamaForCausalLM.from_pretrained(
    base_model,
    load_in_8bit=True,
    device_map="auto",
)
if use_perf:
    model = PeftModel.from_pretrained(
            model,
            '/home/ws/kg2371/lora_weights/llama2_70b_8bit_mof_fold0_4epoch',
            device_map="auto",
    )

# Load paper into variable
file_path = paper_path
lines = read_mmd_file(file_path)
sections = split_mmd_into_sections(lines)

In [ ]:
answer = "Answer (a summarizing paragraph with ALL MOF-related concepts mentioned but excluding everything else. Includes ALL quantities (number of moles) and units (e.g. mmol, min, degC, g, cm, mL, mM, ppm, mg/mL, nm, µL, mol etc.) and exact names (e.g. Girrus sienes, a fruit waste etc.). Special emphasis on composite, MOF, and chemical linker names, metal salt names, as well as enzyme and organism names, number of moles (quantities)  all pH-values of the metal salts, linkers, and enzymes, chemical concentrations, chemical masses, along with everything related to the enzymatic reaction (reactants, products etc.):"

example_input_1 = """# Abstract
In an attempt to make the biocatalyst preparation economically viable, the peroxidase enzyme extracted from orange peel (Girrus sienes, a fruit waste) was immobilized within metal-organic framework (MOF) via self-assembled biomineralization method. The synthesis of OPP-MOF was accomplished by simply mixing 2-methylimidazole (24 mmol), zinc acetate (8 mmol) and orange peel extract at room temperature within 30 min. The obtained OPP-MOF was confirmed by using different characterization methods such as X-ray diffraction (XRD), Fourier transform infrared (FI-IR) and scanning electron microscopy (SEM). The half-life of OPP-MOF was determined in the temperature range of 40-60 degC which showed 2.1 folds enhanced thermal stability as compared to free enzyme. Further, Michaels constant (Ksd) and maximum rate (Vmax) values of OPP-MOF were evaluated by Michaels-Menten kinetics studies which showed higher Ksd and lower Vmax as compared to native peroxidase. Furthermore, peroxidase-MOF retained 48% residual activity after 6th cycle. Also, the storage stability studies of OPP-MOF revealed that there was no significant loss in its activity till 18 days. Finally, immobilized OPP-MOF was used for degradation of methylene blue (MB) and Congo red (CR) dye, and it was found to be more efficient and rapid."""

example_output_1 = f"""{answer}
The MOF was synthesized using the metal salt 2-methylimidazole with a concentration of 24 mmol, the linker salt zinc acetate with a concentration of 8 mmol, and the enzyme orange peel peroxidase (Girrus sienes, a fruit waste). It was named OPP-MOF. The immobilization of peroxidase within MOF was achieved by mixing the enzyme with the MOF components at room temperature for 30 minutes. The half-life of OPP-MOF was determined in the temperatures from 40 to 60 degC which showed 2.1 folds enhanced thermal stability (18 days storage without loss in activity) as compared to free peroxidase. OPP-MOF retained 48% residual activity after 6th cycle. OPP-MOF was used for degradation of methylene blue (MB) and Congo red (CR)."""

example_input_2 = """## Introduction
Enzymes are green catalyst which have evolved from an academic curiosity to industrially attractive technology for driving bio-transformation processes. Also, they are widely used for various applications such as biosensors, bioremediation, wastewater treatment, food and pharmaceutical industries [1,2]. Though enzymes have certain inherent advantages, their applicability on industrial level is always a challenge due to difficulties in recovery and reusability for several cycles. The lack of enzyme stability at industrial operational conditions is also an obstacle in their employment in industry [3]. Another problem associated with the use of soluble enzymes (liquid form) is generation of waste in the reaction mixture. Additionally, it contaminates the product (due to presence of enzyme/protein) which ultimately increases the purification cost [4]. These challenges can be addressed by various enzyme immobilization methods which drastically enhance the enzyme properties such as its thermo-stability, tolerance to organic solvents, wide range of pH, selectivity and its employability in industrial/practical uses. The immobilization also eases the enzyme recovery and re-use, resulting in its cost-effective use focusing its industrial applications [5]. Combined downstream processing includes separation and purification in one step which could be the effective factor to decrease the cost of an enzyme usage on industrial scale. It can be achieved by two ways: (i) by limiting the number of purification steps or simultaneously purifying and (ii) immobilizing the samples. There are some reported strategies for enzyme purification and immobilization in minimum steps, which makes the entire biocatalyst preparation economically viable [6].

Enzymes extraction from fruit wastes can be one of the effective ways of utilization of fruit wastes which are obtained in large quantities (number of moles) during the fruit processing. [15] In India, huge fruit wastes are generated annually which is about 5.6 million ton. These wastes are disposed by dumping on the outskirts [16,17]. Utilizing the waste for value added product formation take us one step further in making the process greener. The peroxidase enzyme from orange peel has been reported to possess wide substrates range, high stability for wide range of pH and temperature. These properties make it an attractive enzyme source for derivation and its valorisation [18]. Earlier, peroxidase immobilization has been reported on various supports such as agarose-chitosan hydrogel [19]. It has been further used for various applications such as environmental remediation (removal of hazardous pollutants, dyes), biomedical, pharmaceutical, nutraceutical, and biotechnological industries [20]. Here, in this work, we have extracted peroxidase from orange peel and immobilized it within MOF, reducing the enzyme purification steps and making it viable for industrial applications. When crude enzyme is mixed with the counterpart of MOFs, enzyme (protein) molecules induce the formation of MOF around enzyme. This results in one step purification and immobilization of OPP into hybrid MOF, forming OPP-MOF composite. Such peroxidase-encapsulated MOF can be readily prepared _via_ a biomineralization procedure in aqueous solution under mild operational conditions. The characterization of OPP-MOF was done by XRD and FT-IR to identify crystalline nature and immobilization of OPP within ZIF-8. For the size, shape and morphology analysis, SEM was used. The Michaelis-Menten kinetics parameters were determined for OPP-MOF and the free form of peroxidase. Thermo-kinetic studies were carried out for free enzyme and OPP-MOF to determine the thermal stability in terms of thermal deactivation constant (k\({}_{\text{d}}\)), half-life (t\({}_{1/2}\)) and deactivation energy (Ea). Finally, different dyes such as methylene blue (MB) and congo red (CR) were degraded by using OPP-MOF."""

example_output_2 = f"""{answer}
The enzyme orange peel peroxidase was extracted and immobilized within MOF. The resulting peroxidase-encapsulated MOF composite (OPP-MOF) has the ability to degrade dyes such as methylene blue (MB) and congo red (CR)."""

example_input_3 = f"""### Preparation of peroxidase extract
Washed orange peels (10 g) were chopped into small pieces (approximately 1 x 1.5 cm) and homogenized into phosphate buffer (0.1 M, pH 7.0) using a blender. The homogenate was filtered through Whatman's filter paper No. 1 and then, filtrant centrifuged (10,000 x (g) at 5 degC) for 20 min. Further, the supernatant was partially purified by 80% ammonium sulphate. The obtained pellet was re-dissolved and used as the peroxidase source for immobilization.
"""

example_output_3 = f"""{answer}
Orange peels (10 g) were chopped into small pieces (~ 1 x 1.5 cm) and homogenized into phosphate buffer (0.1 M, pH 7.0), filtered and centrifuged (10,000 x (g) at 5 degC) for 20min with a Whatman's filter. The supernatant was purified with 80% ammonium sulphate. The obtained pellet was re-dissolved and used as peroxidase source for immobilization.
"""

prompts = []
for section_heading in sections.keys():
    input_text = f'{section_heading}\n{sections[section_heading]}'
    messages = [
        {"role": "system", "content": "You are an expert chemist. You are always truthful and never make facts up. You write short summaries from sections of scientific papers about Metal Organic Frameworks (MOFs). The summaries ALWAYS include ALL MOF-related concepts mentioned in the text but exlcude everything else. Always keep ALL quantities (number of moles) and units (e.g. mmol, min, degC, g, cm, mL, mM, ppm, mg/mL, nm, µL, mol etc.) and exact names (e.g. Girrus sienes, a fruit waste etc.). Always include composite, MOF, and chemical linker names, metal salt names, as well as enzyme and organism names, number of moles (quantities) all pH-values of the metal salts, linkers, and enzymes, chemical concentrations, chemical masses, along with everything related to the enzymatic reaction (reactants, products etc.)."},
        {"role": "user", "content": example_input_1},  # example 1
        {"role": "assistant", "content": example_output_1},  # example 1
        {"role": "user", "content": example_input_2},  # example 1
        {"role": "assistant", "content": example_output_2},  # example 1
        {"role": "user", "content": input_text}
    ]
    prompt = tokenizer.decode(tokenizer.apply_chat_template(messages, return_tensors="pt")[0],skip_special_tokens=False)
    prompt+=f"{answer}\n"
    print(f"Number of tokens for request: {tokenizer(prompt,return_tensors='pt').to('cuda')['input_ids'].shape[1]}")
    prompts.append(prompt)

In [ ]:
from tqdm import tqdm

responses = []
for prompt in tqdm(prompts):
    response = model.generate(**tokenizer(prompt,return_tensors='pt').to('cuda'),do_sample=False, max_new_tokens=2096)
    response = tokenizer.decode(response[0], skip_special_tokens=False)
    response = response.replace(prompt,"")
    response = response.replace(tokenizer.eos_token,"")
    response = response.replace(tokenizer.bos_token,"")
    responses.append(response)

In [ ]:
shorter_text = ''
for i, section_heading in enumerate(sections.keys()):
    shorter_text+=section_heading
    shorter_text+="\n  "
    shorter_text+=responses[i].strip()
    shorter_text+="\n"
print(f"Number of tokens for request: {tokenizer(shorter_text,return_tensors='pt').to('cuda')['input_ids'].shape[1]}")